In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
import seaborn as sns
import re
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
parse_dates = ['issue_d', 'last_pymnt_d', 'earliest_cr_line', 'last_credit_pull_d']
# accepted2 = pd.read_csv('C:\\Users\\nilli\\Downloads\\accepted_b_2015.csv',
#                        low_memory = False, dtype = dtypes, parse_dates = parse_dates)

accepted = pd.read_csv('C:\\Users\\nilli\\Downloads\\accepted_b_2015.csv',
                        parse_dates = parse_dates)


In [ ]:
# Features engineering


In [3]:
accepted["term"] = accepted["term"].apply(lambda x: re.sub('months', '',x)).astype('int64')  

In [4]:
accepted["emp_length"] = accepted['emp_length'].apply(lambda x: re.sub('years', '',x))#.astype('int64')
accepted["emp_length"] = accepted['emp_length'].apply(lambda x: re.sub('year', '',x))
accepted["emp_length"] = accepted['emp_length'].apply(lambda x: re.sub('< 1 ', '0',x))
accepted["emp_length"] = accepted["emp_length"].str.replace(r'\D', '', ).astype('int64')

In [5]:
grade_ord = {'A': 7,'B': 6,'C': 5,'D': 4,'E': 3,'F': 2,'G': 1}
accepted['grade']= accepted['grade'].map(grade_ord)   

In [6]:
post_loan_columns = ['verification_status', 'loan_status', 'out_prncp',
                     'total_pymnt','total_rec_prncp','total_rec_int',
                     'total_rec_late_fee', 'recoveries', 
                     'collection_recovery_fee', 'last_pymnt_d',
                     'last_pymnt_amnt','last_credit_pull_d', 
                     'last_fico_range_high','last_fico_range_low',
                     'duration_days','duration_months','invest_return_per',
                     'invest_return']

In [7]:
#Remove from features
remove_columns = ['issue_d', 'id','earliest_cr_line','addr_state', 'initial_list_status']


#Count columns
counting_columns = ['delinq_2yrs','inq_last_6mths', 'open_acc', 
                    'pub_rec', 'total_acc', 'collections_12_mths_ex_med', 
                    'acc_now_delinq', 'chargeoff_within_12_mths', 
                    'mort_acc','num_accts_ever_120_pd', 'num_actv_bc_tl', 
                    'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
                    'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
                    'num_tl_120dpd_2m', 'num_tl_30dpd','num_tl_90g_dpd_24m', 
                    'num_tl_op_past_12m', 'pub_rec_bankruptcies', 'tax_liens',
                    'acc_open_past_24mths', 'num_actv_rev_tl']


# Categorical
cat_columns = ['term', 'home_ownership', 'purpose']# removing 'addr_state', 'initial_list_status']

#Ordinal
ordinal_columns = ['grade', 'sub_grade', 'emp_length']

#earliest_cr_line is a datetime column. The rest is the number of months since something
time_columns = ['mo_sin_old_rev_tl_op', 'mo_sin_rcnt_tl', 
                'mths_since_recent_bc','mo_sin_rcnt_rev_tl_op'] # removing,'earliest_cr_line']

non_continuous_columns = counting_columns + cat_columns + ordinal_columns + time_columns
continuous_columns = ['loan_amnt','int_rate',
                      'installment','annual_inc','dti',
                      'fico_range_low','fico_range_high',
                      'revol_bal','revol_util','tot_coll_amt',
                      'tot_cur_bal','total_rev_hi_lim',
                      'avg_cur_bal','bc_open_to_buy','bc_util',
                      'delinq_amnt','pct_tl_nvr_dlq',
                      'percent_bc_gt_75','tot_hi_cred_lim',
                      'total_bal_ex_mort','total_bc_limit',
                      'total_il_high_credit_limit'] # removing 'funded_amnt',

feature_columns = continuous_columns + non_continuous_columns

In [8]:
import category_encoders as ce

sub_grades = sorted(accepted['sub_grade'].unique())
sub_grade_map = {e:i for i, e in enumerate(sub_grades)}

emp_lengths = sorted(accepted['emp_length'].unique())
emp_length_map = {e:i for i, e in zip([1, 10, 2, 3, 4, 5, 6, 7, 8, 9, 0], emp_lengths)}


mapping=[{'col':'sub_grade' , 'mapping':sub_grade_map}, 
         {'col':'emp_length', 'mapping':emp_length_map}]

for col in cat_columns:
    col_map = {e:i for i, e in enumerate(sorted(accepted[col].unique()))}
    mapping.append({'col':col, 'mapping': col_map})

# dropping grade since it is redundant with sub_grade
# accepted.drop(columns = 'grade', inplace = True)
# ordinal_columns.remove('grade')
# feature_columns.remove('grade')
# non_continuous_columns.remove('grade')


# # dropping sub_grade since it is redundant with grade
# accepted.drop(columns = 'sub_grade', inplace = True)
# ordinal_columns.remove('sub_grade')
# feature_columns.remove('sub_grade')
# non_continuous_columns.remove('sub_grade')


In [9]:
loan_status_cat_ord = {'Defaulted':0, 'FullyPaid':1}
accepted['loan_status'] = accepted['loan_status'].map(loan_status_cat_ord) 

In [10]:
encoder = ce.OrdinalEncoder(cols= cat_columns + ordinal_columns, 
                            mapping=mapping)

Y = accepted['loan_status']

encoder.fit(accepted, Y)

encoder.transform(accepted)[feature_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453776 entries, 0 to 453775
Data columns (total 58 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   loan_amnt                   453776 non-null  float64
 1   int_rate                    453776 non-null  float64
 2   installment                 453776 non-null  float64
 3   annual_inc                  453776 non-null  float64
 4   dti                         453776 non-null  float64
 5   fico_range_low              453776 non-null  float64
 6   fico_range_high             453776 non-null  float64
 7   revol_bal                   453776 non-null  float64
 8   revol_util                  453776 non-null  float64
 9   tot_coll_amt                453776 non-null  float64
 10  tot_cur_bal                 453776 non-null  float64
 11  total_rev_hi_lim            453776 non-null  float64
 12  avg_cur_bal                 453776 non-null  float64
 13  bc_open_to_buy

In [12]:
X = accepted[feature_columns] 
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,fico_range_high,revol_bal,revol_util,tot_coll_amt,...,term,home_ownership,purpose,grade,sub_grade,emp_length,mo_sin_old_rev_tl_op,mo_sin_rcnt_tl,mths_since_recent_bc,mo_sin_rcnt_rev_tl_op
0,10400.0,6.99,321.08,58000.0,14.92,710.0,714.0,6133.0,31.6,0.0,...,36,MORTGAGE,credit_card,7,A3,8,290.0,1.0,5.0,1.0
1,15000.0,12.39,336.64,78000.0,12.03,750.0,754.0,138008.0,29.0,0.0,...,60,RENT,debt_consolidation,5,C1,10,244.0,1.0,47.0,1.0
2,9600.0,13.66,326.53,69000.0,25.81,680.0,684.0,16388.0,59.4,0.0,...,36,RENT,debt_consolidation,5,C3,10,265.0,3.0,24.0,23.0
3,7650.0,13.66,260.20,50000.0,34.81,685.0,689.0,16822.0,91.9,0.0,...,36,RENT,debt_consolidation,5,C3,0,148.0,8.0,17.0,8.0
4,21425.0,15.59,516.36,63800.0,18.49,685.0,689.0,16374.0,76.2,0.0,...,60,RENT,credit_card,4,D1,6,136.0,7.0,7.0,7.0


In [16]:
accepted.select_dtypes(include = 'object').columns

Index(['sub_grade', 'home_ownership', 'verification_status', 'purpose',
       'addr_state', 'initial_list_status'],
      dtype='object')

In [44]:
X= X.drop('sub_grade', axis =1)
X

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,fico_range_high,revol_bal,revol_util,tot_coll_amt,...,num_actv_rev_tl,term,home_ownership,purpose,grade,emp_length,mo_sin_old_rev_tl_op,mo_sin_rcnt_tl,mths_since_recent_bc,mo_sin_rcnt_rev_tl_op
0,10400.0,6.99,321.08,58000.0,14.92,710.0,714.0,6133.0,31.6,0.0,...,9.0,36,MORTGAGE,credit_card,7,8,290.0,1.0,5.0,1.0
1,15000.0,12.39,336.64,78000.0,12.03,750.0,754.0,138008.0,29.0,0.0,...,4.0,60,RENT,debt_consolidation,5,10,244.0,1.0,47.0,1.0
2,9600.0,13.66,326.53,69000.0,25.81,680.0,684.0,16388.0,59.4,0.0,...,7.0,36,RENT,debt_consolidation,5,10,265.0,3.0,24.0,23.0
3,7650.0,13.66,260.20,50000.0,34.81,685.0,689.0,16822.0,91.9,0.0,...,4.0,36,RENT,debt_consolidation,5,0,148.0,8.0,17.0,8.0
4,21425.0,15.59,516.36,63800.0,18.49,685.0,689.0,16374.0,76.2,0.0,...,4.0,60,RENT,credit_card,4,6,136.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453771,20500.0,16.77,728.54,60000.0,16.40,700.0,704.0,15417.0,58.4,0.0,...,6.0,36,RENT,debt_consolidation,4,7,155.0,5.0,13.0,7.0
453772,15000.0,15.27,521.97,57600.0,8.35,680.0,684.0,8897.0,33.1,0.0,...,5.0,36,RENT,small_business,5,3,163.0,6.0,14.0,8.0
453773,35000.0,15.96,1229.81,160000.0,4.90,720.0,724.0,23665.0,62.4,0.0,...,5.0,36,MORTGAGE,small_business,5,3,162.0,5.0,14.0,8.0
453774,12000.0,16.29,423.61,35000.0,12.93,675.0,679.0,15006.0,93.2,0.0,...,5.0,36,MORTGAGE,other,4,5,158.0,5.0,13.0,7.0


In [45]:
from sklearn.preprocessing import OneHotEncoder

#dummify purpose column
purpose_ohe = OneHotEncoder( drop='first', sparse = False )
purpose_enc = purpose_ohe.fit_transform( X[['purpose']] )
purpose_enc = pd.DataFrame( purpose_enc, columns=purpose_ohe.get_feature_names(['purpose']) )
X= pd.concat( (X.drop(["purpose"], axis=1).reset_index(drop = True), purpose_enc), axis = 1 )

#dummify home_ownership column 
home_ownership_ohe = OneHotEncoder( drop='first', sparse = False )
home_ownership_enc = home_ownership_ohe.fit_transform( X[['home_ownership']] )
home_ownership_enc = pd.DataFrame( home_ownership_enc, columns=home_ownership_ohe.get_feature_names(['home_ownership']) )
X= pd.concat( (X.drop(["home_ownership"], axis=1).reset_index(drop = True), home_ownership_enc), axis = 1 )


In [49]:
#X = accepted[feature_columns]
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,fico_range_high,revol_bal,revol_util,tot_coll_amt,...,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,10400.0,6.99,321.08,58000.0,14.92,710.0,714.0,6133.0,31.6,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,15000.0,12.39,336.64,78000.0,12.03,750.0,754.0,138008.0,29.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,9600.0,13.66,326.53,69000.0,25.81,680.0,684.0,16388.0,59.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,7650.0,13.66,260.20,50000.0,34.81,685.0,689.0,16822.0,91.9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,21425.0,15.59,516.36,63800.0,18.49,685.0,689.0,16374.0,76.2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:
X.shape

(453776, 73)

In [53]:
Y =accepted['loan_status']


In [54]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split( X,Y, test_size = 0.2 ,random_state=42,stratify =Y )
print(Xtrain.shape)
print(ytrain.shape)

(363020, 73)
(363020,)


In [55]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
Xtrain = pd.DataFrame( ss.fit_transform(Xtrain), columns = Xtrain.columns)
Xtrain.head()
Xtest = pd.DataFrame( ss.fit_transform(Xtest), columns = Xtest.columns)
Xtest.head()

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,fico_range_high,revol_bal,revol_util,tot_coll_amt,...,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,1.313133,0.487780,0.717277,-0.056347,-0.890618,0.775515,0.775506,0.407950,-0.162346,-0.100073,...,-0.230722,-0.029703,-0.12467,-0.076351,-0.071919,0.992098,-0.01101,-0.01992,-0.312365,-0.827801
1,0.707241,0.920488,1.183959,0.703055,-0.616517,-1.173989,-1.173980,0.191004,1.493740,-0.100073,...,-0.230722,-0.029703,-0.12467,-0.076351,-0.071919,-1.007965,-0.01101,-0.01992,-0.312365,1.208020
2,-0.504543,-0.941537,-0.447409,2.981261,1.895012,0.613056,0.613049,2.604269,1.337824,-0.100073,...,-0.230722,-0.029703,-0.12467,-0.076351,-0.071919,0.992098,-0.01101,-0.01992,-0.312365,-0.827801
3,0.707241,1.429150,0.403637,0.075789,1.971506,0.125680,0.125677,0.707300,1.401033,-0.100073,...,-0.230722,-0.029703,-0.12467,-0.076351,-0.071919,0.992098,-0.01101,-0.01992,-0.312365,-0.827801
4,-0.746899,0.469367,-0.613436,-0.568943,1.689755,-0.199237,-0.199237,-0.748350,1.063916,-0.100073,...,-0.230722,-0.029703,-0.12467,-0.076351,-0.071919,0.992098,-0.01101,-0.01992,-0.312365,-0.827801


In [85]:
from sklearn. linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
logit = LogisticRegression( class_weight='balanced')
# logit = LogisticRegression( )
#logit = LogisticRegression( class_weight='balanced',multi_class='ovr',solver='liblinear')
param_grid={"C":np.logspace(-2,3,7)}
#param_grid={'C': [100]}
#param_grid={"C":np.linspace(10,1000,20)}
logit_cv=GridSearchCV(logit, param_grid, cv =5, n_jobs = -1)
logit_cv.fit(Xtrain,ytrain)


# C=1.0, class_weight=None, dual=False, fit_intercept=True,
#           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
#           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
#           verbose=0, warm_start=False
            
           # solver='lbfgs',

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                          dual=False, fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-02, 6.81292069e-02, 4.64158883e-01, 3.16227766e+00,
       2.15443469e+01, 1.46779927e+02, 1.00000000e+03])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [57]:
#best hyperparemeter
logit_cv.best_params_ 


{'C': 21.544346900318846}

In [58]:
from sklearn.metrics import classification_report, roc_auc_score

In [59]:
roc=roc_auc_score(ytest, logit_cv.predict_proba(Xtest)[:,1])
roc

0.7055234378287152

In [61]:
logit_cv.predict_proba(Xtest)

array([[0.18591087, 0.81408913],
       [0.15336099, 0.84663901],
       [0.0312948 , 0.9687052 ],
       ...,
       [0.12215447, 0.87784553],
       [0.15320325, 0.84679675],
       [0.03486028, 0.96513972]])

In [62]:
#best score
logit_cv.best_score_

0.8289074982094651

In [63]:
logit_cv.score(Xtrain,ytrain)

0.8289322902319431

In [64]:
logit_cv.score(Xtest,ytest)

0.8291573008947067

In [65]:
from sklearn.metrics import confusion_matrix,classification_report
confusion_matrix(ytest, logit_cv.predict(Xtest))

array([[  564, 14952],
       [  553, 74687]], dtype=int64)

In [66]:
print(classification_report(ytest, logit_cv.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.50      0.04      0.07     15516
           1       0.83      0.99      0.91     75240

    accuracy                           0.83     90756
   macro avg       0.67      0.51      0.49     90756
weighted avg       0.78      0.83      0.76     90756



To handle the imbalance in the dataset,the class weight chosen was 'balanced' in the logistic regression model.
This implies the sampling of the small class to match the weight of the bigger class. We could also downsample the majority class

In [67]:
accepted['loan_status'].value_counts()

1    376195
0     77581
Name: loan_status, dtype: int64

In [68]:
X.columns

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'fico_range_low', 'fico_range_high', 'revol_bal', 'revol_util',
       'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'avg_cur_bal',
       'bc_open_to_buy', 'bc_util', 'delinq_amnt', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bal_ex_mort',
       'total_bc_limit', 'total_il_high_credit_limit', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc',
       'collections_12_mths_ex_med', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'mort_acc', 'num_accts_ever_120_pd',
       'num_actv_bc_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl',
       'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
       'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m',
       'num_tl_op_past_12m', 'pub_rec_bankruptcies', 'tax_liens',
       'acc_open_past_24mths', 'num_actv_rev_tl', 'term', 'grade',
       'emp_length', 'mo_sin_old_rev_tl_op', 'mo_sin_r

In [69]:
X.shape

(453776, 73)

In [70]:
df = pd.DataFrame(accepted['loan_status'])
df.head()

,loan_status
0,0
1,1
2,1
3,0
4,1


In [71]:
#df2 = pd.concat([X, df])
df2= pd.merge(X,df,left_on=None,left_index=True, right_index=True)
df2.head()

# df2 = pd.merge(X, accepted[accepted['loan_status']])

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,fico_range_high,revol_bal,revol_util,tot_coll_amt,...,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,loan_status
0,10400.0,6.99,321.08,58000.0,14.92,710.0,714.0,6133.0,31.6,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
1,15000.0,12.39,336.64,78000.0,12.03,750.0,754.0,138008.0,29.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,9600.0,13.66,326.53,69000.0,25.81,680.0,684.0,16388.0,59.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
3,7650.0,13.66,260.20,50000.0,34.81,685.0,689.0,16822.0,91.9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,21425.0,15.59,516.36,63800.0,18.49,685.0,689.0,16374.0,76.2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [72]:
from sklearn.utils import resample
#Separate majority and minority classes
df_majority = df2[df2['loan_status']==1]
df_minority = df2[df2['loan_status']==0]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=77581,     # to match minority class
                                 random_state=123) 
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.loan_status.value_counts()
# 1    77581
# 0    77581
# Name: loan_status, dtype: int64

1    77581
0    77581
Name: loan_status, dtype: int64

In [75]:
Y2 =df_downsampled['loan_status']
Y2.shape

(155162,)

In [76]:
X2 = df_downsampled.drop('loan_status', axis =1)
X2.shape

(155162, 73)

In [77]:
from sklearn.model_selection import train_test_split

X2train, X2test, y2train, y2test = train_test_split( X2,Y2, test_size = 0.2 ,random_state=42)
print(X2train.shape)
print(y2train.shape)

(124129, 73)
(124129,)


In [78]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X2train = pd.DataFrame( ss.fit_transform(X2train), columns = X2train.columns)
X2train.head()
X2test = pd.DataFrame( ss.fit_transform(X2test), columns = X2test.columns)
X2test.head()

,loan_amnt,int_rate,installment,annual_inc,dti,fico_range_low,fico_range_high,revol_bal,revol_util,tot_coll_amt,...,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,-0.014916,-1.415728,0.073672,-0.795586,-0.467439,0.756555,0.756533,-0.502658,-1.240358,-0.096623,...,-0.239914,-0.02782,-0.134075,-0.071536,-0.068039,-0.961772,-0.013906,-0.021245,-0.316093,1.156058
1,-1.098245,-0.828929,-1.065761,-0.186895,-1.779189,-1.136028,-1.136006,-0.554658,0.708561,2.005758,...,-0.239914,-0.02782,-0.134075,-0.071536,-0.068039,-0.961772,-0.013906,-0.021245,-0.316093,1.156058
2,0.647119,0.146059,0.164429,0.295860,-0.300144,-0.103710,-0.103712,0.011717,0.191414,-0.096623,...,-0.239914,-0.02782,-0.134075,-0.071536,-0.068039,1.039748,-0.013906,-0.021245,-0.316093,-0.865008
3,-1.396161,0.256648,-1.370120,-0.702393,-0.110035,-0.447816,-0.447810,-0.510820,0.601712,-0.096623,...,-0.239914,-0.02782,-0.134075,-0.071536,-0.068039,-0.961772,-0.013906,-0.021245,-0.316093,1.156058
4,0.887859,0.588415,0.456116,-0.438767,0.258776,0.240396,0.240386,-0.294011,-0.787320,-0.096623,...,-0.239914,-0.02782,-0.134075,-0.071536,-0.068039,1.039748,-0.013906,-0.021245,-0.316093,-0.865008


In [79]:
from sklearn. linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
#logit = LogisticRegression( class_weight='balanced')
logit = LogisticRegression( )
#logit = LogisticRegression( class_weight='balanced',multi_class='ovr',solver='liblinear')
#param_grid={"C":np.logspace(-3,3,7)}
param_grid={"C":np.linspace(0.01,49,40)}
#param_grid={'C': [308.3846153846154]}
#param_grid={"C":np.linspace(10,1000,20)}
logit_cv=GridSearchCV(logit, param_grid, cv =5, n_jobs = -1)
logit_cv.fit(X2train,y2train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-02, 1.266...
       2.51330769e+01, 2.63892308e+01, 2.76453846e+01, 2.89015385e+01,
       3.01576923e+01, 3.14138462e+01, 3.26700000e+01, 3.39261538e+01,
       3.51823077e+01, 3.64384615e+01, 3.76946154e+01, 3.89507692e+01,
       4.02069231e+01, 4.14630769e+01, 4.27192308e+01, 4.39753846e+01,
       4

In [80]:
logit_cv.best_params_ 

{'C': 8.803076923076922}

In [81]:
from sklearn.metrics import classification_report, roc_auc_score

In [82]:
roc2=roc_auc_score(y2test, logit_cv.predict_proba(X2test)[:,1])
roc2

0.7047756308348184

In [83]:
from sklearn.metrics import confusion_matrix,classification_report
confusion_matrix(y2test, logit_cv.predict(X2test))

array([[10033,  5542],
       [ 5280, 10178]], dtype=int64)

In [84]:
print(classification_report(y2test, logit_cv.predict(X2test)))

              precision    recall  f1-score   support

           0       0.66      0.64      0.65     15575
           1       0.65      0.66      0.65     15458

    accuracy                           0.65     31033
   macro avg       0.65      0.65      0.65     31033
weighted avg       0.65      0.65      0.65     31033

